In [ ]:
#Housing Case Study !!!

**Housing Case Study**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv('../input/housing-simple-regression/Housing.csv')

In [ ]:
data.head()

In [ ]:
#What type of values stored in columns...
data.info()

In [ ]:
data['furnishingstatus'].value_counts()

**DATA PREPARATION**

In [ ]:
#Converting Yes to 1 and No to 0
data['mainroad']=data['mainroad'].map({'yes':1,'no':0})
data['guestroom']=data['guestroom'].map({'yes':1,'no':0})
data['basement']=data['basement'].map({'yes':1,'no':0})
data['hotwaterheating']=data['hotwaterheating'].map({'yes':1,'no':0})
data['airconditioning']=data['airconditioning'].map({'yes':1,'no':0})
data['prefarea']=data['prefarea'].map({'yes':1,'no':0})

In [ ]:
data.head()

In [ ]:
status=pd.get_dummies(data['furnishingstatus'],drop_first=True)

In [ ]:
status.head()

In [ ]:
data=pd.concat([data,status],axis=1)

In [ ]:
data.head()

In [ ]:
data.drop(['furnishingstatus'],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
#CREATING NEW VARIABLES..
data['areaperbedroom']=data['area']/data['bedrooms']


In [ ]:
data['bbratio']=data['bathrooms']/data['bedrooms']

In [ ]:
data.head()

In [ ]:
def normalize(x):
    return ((x-np.min(x))/(max(x)-min(x)))


In [ ]:
#Applying Normalize to all columns

In [ ]:
data=data.apply(normalize)

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
X=data[['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'hotwaterheating', 'airconditioning',
       'parking', 'prefarea', 'semi-furnished', 'unfurnished',
       'areaperbedroom', 'bbratio']]
y=data['price']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=100)

In [ ]:
import statsmodels.api as sm
X_train=sm.add_constant(X_train)
lm_l=sm.OLS(y_train,X_train).fit()

In [ ]:
print(lm_l.summary())

In [ ]:
# # Variables might be correlated with each  other...
# # What about if we have more than 1 columns which is highly correlated
# #Correlation matrix can only work for 2 variables but for not more than 2


# We will Use VIF (Variance  Inflation Factor)
# VIF(x)=1/(1-R^2)
#if R^2 Higher VIF also Higher 
#Higher the VIF Higher the muticollinearity   
#Variable with Higher VIF may not be statistically significant


In [ ]:
#FUNCTION FOR VIF

In [ ]:
def vif_cal(input_data,dependent_col):
    vif_df=pd.DataFrame(columns=['Var','Vif'])
    x_vars=input_data.drop([dependent_col],axis=1)
    x_var_names=x_vars.columns
    for i in range(0,x_var_names.shape[0]):
        y=x_vars[x_var_names[i]]
        x=x_vars[x_var_names.drop(x_var_names[i])]
        rsq=sm.OLS(y,x).fit().rsquared
        vif=round(1/(1-rsq),2)
        vif_df.loc[i]=[x_var_names[i],vif]
    return vif_df.sort_values(by='Vif',axis=0,ascending=False,inplace=False)

In [ ]:
vif_cal(input_data=data,dependent_col="price")

In [ ]:
# REMOVE BB RATIO BECAUSE IT HAS HIGH P-value and high VIF

In [ ]:
#CORRELATION MATRIX

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(data.corr(),annot=True)

In [ ]:
#Dropping The variable and Updating The model

In [ ]:
X_train=X_train.drop(['bbratio'],axis=1)

In [ ]:
lm_2=sm.OLS(y_train,X_train).fit()

In [ ]:
print(lm_2.summary())

In [ ]:
vif_cal(input_data=data.drop(['bbratio'],axis=1),dependent_col="price")

In [ ]:
#DROPPING BEDROOM ALSO
X_train=X_train.drop(['bedrooms'],axis=1)

In [ ]:
lm_3=sm.OLS(y_train,X_train).fit()

In [ ]:
print(lm_3.summary())

In [ ]:
vif_cal(input_data=data.drop(['bbratio','bedrooms'],axis=1),dependent_col="price")

In [ ]:
#Removing Area PEr Bedroom

In [ ]:
X_train=X_train.drop(['areaperbedroom'],axis=1)

In [ ]:
lm_4=sm.OLS(y_train,X_train).fit()

In [ ]:
lm_4.summary()

In [ ]:
vif_cal(input_data=data.drop(['bbratio','bedrooms','areaperbedroom'],axis=1),dependent_col="price")

In [ ]:
#REMOVING SEMI-FURNISHED ALSO

In [ ]:
X_train.drop(['semi-furnished','basement'],axis=1,inplace=True)

In [ ]:
lm_5=sm.OLS(y_train,X_train).fit()

In [ ]:
print(lm_5.summary())

In [ ]:
vif_cal(input_data=data.drop(['bbratio','bedrooms','areaperbedroom','semi-furnished','basement'],axis=1),dependent_col="price")

In [ ]:
# DESIGN model by dropping all variables one by one with high VIF. Then compare results

In [ ]:
X_test_m5=sm.add_constant(X_test)

In [ ]:
X_test_m5=X_test_m5.drop(['bbratio','bedrooms','areaperbedroom','semi-furnished','basement'],axis=1)

In [ ]:
y_pred_m5=lm_5.predict(X_test_m5)

In [ ]:
#MODEL EVALUATION

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
mse=mean_squared_error(y_test,y_pred_m5)
rsq=r2_score(y_test,y_pred_m5)

In [ ]:
mse

In [ ]:
rsq

In [ ]:
#We can also use RFE...